In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

/Users/zhanglinzi/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
convlayer = torch.nn.Conv1d(1,256,5)

In [3]:
x = torch.randint(1,10,(1,20)).type(torch.float)

In [4]:
print(x)

tensor([[8., 7., 9., 1., 4., 3., 6., 5., 4., 3., 1., 8., 6., 6., 4., 2., 4., 4.,
         8., 3.]])


In [5]:
print(x.unsqueeze(0).shape)

torch.Size([1, 1, 20])


In [6]:
## CausalConv1d Block

class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super(CausalConv1d, self).forward(F.pad(input, (self.__padding, 0)))

In [7]:
## Dilated-casual CNN Architecture

class DilatedCNN(nn.Module):
    
    def __init__(self,input_channels,output_channels,kernel_size):
        super(DilatedCNN, self).__init__()

        self.dil1 = CausalConv1d(in_channels=input_channels,out_channels=32,kernel_size=kernel_size,
                                dilation=1)
        self.dil2 = CausalConv1d(in_channels=32,out_channels=16,kernel_size=kernel_size,
                                dilation=2)
        self.dil3 = CausalConv1d(in_channels=16,out_channels=output_channels,kernel_size=kernel_size,
                                dilation=4)
        
#         self.fc1 = nn.Linear(20,1)
#         self.drop1 = nn.Dropout(0.2)
#         self.fc2 = nn.Linear(8,1)
    
    def forward(self, x):
        
        # for predicting y0(y at timestep 0), we need to use
        # x-value at the last timestep.
#         x_now = x.clone()[:,:,-1]
        x = torch.relu(self.dil1(x))
        x = torch.relu(self.dil2(x))
        x = torch.relu(self.dil3(x))
    
        
#         x = x[:,:,-1]
        # batch_size, features, time_steps -> batch_size, features
        # take only the last timestep
        
#         x = torch.cat([x,x_now], axis=1)
#         x = F.relu(self.fc1(x))
#         x = self.drop1(x)
#         x = self.fc2(x)

        return x
        

In [8]:
input_channels=1
output_channels=1
kernel_size = 5

In [10]:
convlayer = DilatedCNN(input_channels,output_channels,kernel_size)

In [11]:
print(convlayer(x.unsqueeze(0)).shape)
print(convlayer(x.unsqueeze(0)))

torch.Size([1, 1, 20])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.2696, 0.3935, 0.2615, 0.0000,
          0.3183, 0.3757, 0.4349, 0.0006]]], grad_fn=<ReluBackward0>)


In [18]:
class context_embedding(torch.nn.Module):
    def __init__(self,input_channels=1,embedding_size=256,k=5):
        super(context_embedding,self).__init__()
        self.dilated_causal_convolution = DilatedCNN(input_channels,embedding_size,kernel_size)

    def forward(self,x):
        x = self.dilated_causal_convolution(x)
        return F.sigmoid(x)

In [19]:
embedding = context_embedding(1,256,5)

In [20]:
embedding(x.unsqueeze(0)).shape

/Users/zhanglinzi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([1, 256, 20])